# Jigsaw - Agile Community Rules Classification
### https://www.kaggle.com/competitions/jigsaw-agile-community-rules

In [13]:
#!/usr/bin/python3
!which python3

/usr/bin/python3


In [14]:
!pwd

/data


In [15]:
%%bash
set -e  # exit immediately if a command fails

VENV_DIR="myvenv"

# Step 1: Ensure uv is installed globally to create venv
if ! command -v uv &> /dev/null; then
  echo "Installing uv globally to create venv..."
  python3 -m ensurepip --upgrade || true
  python3 -m pip install --upgrade pip setuptools wheel
  python3 -m pip install --upgrade uv
else
  echo "✅ uv is already installed globally."
fi

# Step 2: Create virtual environment if it doesn't exist
if [ -d "$VENV_DIR" ]; then
  echo "✅ Virtual environment '$VENV_DIR' already exists. Skipping creation."
else
  echo "Creating virtual environment with uv..."
  uv venv "$VENV_DIR"
fi

# Step 3: Activate virtual environment
echo "Activating virtual environment..."
source "$VENV_DIR/bin/activate"

# Step 4: Upgrade pip, setuptools, wheel, and uv *inside* the venv
echo "Upgrading pip, setuptools, wheel, and uv inside the venv..."
python3 -m ensurepip --upgrade || true
python3 -m pip install --upgrade pip setuptools wheel --progress-bar=on
python3 -m pip install --upgrade uv --progress-bar=on

echo "Checking Python binary path:"
which python3

echo "Checking Python version:"
python3 --version

echo "Checking uv version inside venv:"
uv --version

echo "✅ Virtual environment setup complete and up to date!"


✅ uv is already installed globally.
✅ Virtual environment 'myvenv' already exists. Skipping creation.
Activating virtual environment...
Upgrading pip, setuptools, wheel, and uv inside the venv...
Looking in links: /tmp/tmpea0z1jbk
Checking Python binary path:
/data/myvenv/bin/python3
Checking Python version:
Python 3.11.13
Checking uv version inside venv:
uv 0.8.18
✅ Virtual environment setup complete and up to date!


In [16]:
%%bash
set -e  # Exit immediately if a command fails

PYTHON=./myvenv/bin/python

# echo "Bootstrapping pip if needed..."
# $PYTHON -m ensurepip --upgrade || true
# $PYTHON -m pip install --upgrade pip setuptools wheel

# echo "Installing uv..."
# $PYTHON -m pip install --upgrade uv

echo "Installing required libraries with uv..."
$PYTHON -m uv pip install \
  trl \
  optimum \
  auto-gptq \
  bitsandbytes \
  peft \
  accelerate \
  deepspeed \
  kagglehub

echo "✅ Done installing packages into myvenv using uv."


Installing required libraries with uv...


Using Python 3.11.13 environment at: myvenv
Audited 8 packages in 79ms


✅ Done installing packages into myvenv using uv.


In [17]:
%%writefile config.py

RESUME_TRAINING=False
#LOCAL_MODEL_PATH = "Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4"
LOCAL_MODEL_PATH = "Qwen/Qwen2.5-3B-Instruct"
LORA_IN_PATH= "/workspace/lora"
LORA_OUT_PATH = "/workspace"
DATA_PATH = "./"
OUTPUT_PATH="/workspace"

# Training parameters
MAX_SEQ_LENGTH = 3072
RANK = 32
LORA_ALPHA=32
MAX_ITER_STEPS = -1
EPOCHS = 2
SAMPLE_LEN="35k"

# Kaggle upload configuration
MODEL_SLUG = "qwen25-3b-instruct-jigsaw-acrc-lora"
VARIATION_SLUG = "17"

###--------------------------------###
DATASET_ID="0-swap-cr12-kdsr1"
BASE_MODEL=LOCAL_MODEL_PATH.split("/")[-1].replace(".", "p")
TRAIN_DIR=f"{BASE_MODEL}_lora_fp16_r{RANK}_s{SAMPLE_LEN}_e_{EPOCHS}_msl{MAX_SEQ_LENGTH}-{DATASET_ID}"
print("TRAIN_DIR",TRAIN_DIR)

Overwriting config.py


In [ ]:
%%writefile get_dataset.py
import pandas as pd
from datasets import Dataset
import kagglehub
import os
import glob

def load_data():
    """Load Jigsaw ACRC dataset from Kaggle or local files"""
    # Check if running on Kaggle
    if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
        # Running on Kaggle
        base_path = "/kaggle/input/jigsaw-agile-community-rules/"
        df_train = pd.read_csv(f"{base_path}*train*.csv")
        df_test = pd.read_csv(f"{base_path}*test*.csv")
    else:
        # Running locally
        base_path = "./"
        
        # Find all train files
        train_files = glob.glob(f"{base_path}*train*.csv")
        if train_files:
            train_dfs = [pd.read_csv(file) for file in train_files]
            df_train = pd.concat(train_dfs, ignore_index=True)
            print(f"Concatenated {len(train_files)} train files: {train_files}")
        else:
            raise FileNotFoundError(f"No train files found in {base_path}")
        
        # Find all test files
        test_files = glob.glob(f"{base_path}*test*.csv")
        if test_files:
            test_dfs = [pd.read_csv(file) for file in test_files]
            df_test = pd.concat(test_dfs, ignore_index=True)
            print(f"Concatenated {len(test_files)} test files: {test_files}")
        else:
            raise FileNotFoundError(f"No test files found in {base_path}")

    print(f"Train shape: {df_train.shape}")
    print(f"Test shape: {df_test.shape}")
    print(df_train.columns)
            
    req_cols=['subreddit', 'rule', 'positive_example_1', 'negative_example_1', 'positive_example_2',
           'negative_example_2', 'test_comment', 'violates_rule']

    df_train=df_train[req_cols]
    df_test=df_test[req_cols]

    # Normalize "True"/"False" -> "Yes"/"No" and drop anything else
    for name, df in [("train", df_train), ("test", df_test)]:
        df["violates_rule"] = (
            df["violates_rule"]
            .astype(str).str.strip()
            .map({"True": "Yes", "False": "No", "Yes": "Yes", "No": "No"})  # normalize
        )
        before = len(df)
        df.dropna(subset=["violates_rule"], inplace=True)  # drop rows with NaN (anything not Yes/No/True/False)
        after = len(df)
        print(f"Dropped {before - after} rows from {name} due to invalid 'violates_rule'")
    
    for col in req_cols:
        dropped_rows = df_train[df_train[col].isna()].shape[0]
        print(f"{col}: {dropped_rows} rows would be dropped")
        
    df_train = df_train[req_cols].dropna()
    df_test = df_test[req_cols].dropna()


    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)

    print(f"Using path: {base_path}")
    print("\n After dropping:")
    print(f"Train shape: {df_train.shape}")
    print(f"Test shape: {df_test.shape}")

    df_train["violates_rule"] = df_train["violates_rule"].astype(str)
    df_test["violates_rule"] = df_test["violates_rule"].astype(str)

    valid_values = {"Yes", "No"}
    df_train = df_train[df_train["violates_rule"].isin(valid_values)]
    df_test  = df_test[df_test["violates_rule"].isin(valid_values)]
    print("\n After checking Yes/No:")
    print(f"Train shape: {df_train.shape}")
    print(f"Test shape: {df_test.shape}")
    
    return df_train, df_test

def formatting_prompts_func(examples, tokenizer):
    """
    Format Reddit moderation dataset using tokenizer chat template
    """
    texts = []
    for i in range(len(examples['subreddit'])):
        # Create system message
        system_msg = f"""You are a really experienced moderator for the subreddit /r/{examples['subreddit'][i]}. 
Your job is to determine if the following reported comment violates the given rule. Answer with only Yes or No."""
        
        # Create user message with the rule and examples
        user_msg = f"""<rule>
{examples['rule'][i]}
</rule>

<examples>
<example>
<comment>{examples['positive_example_1'][i]}</comment>
<rule_violation>Yes</rule_violation>
</example>

<example>
<comment>{examples['positive_example_2'][i]}</comment>
<rule_violation>Yes</rule_violation>
</example>

<example>
<comment>{examples['negative_example_1'][i]}</comment>
<rule_violation>No</rule_violation>
</example>

<example>
<comment>{examples['negative_example_2'][i]}</comment>
<rule_violation>No</rule_violation>
</example>
</examples>

<test_comment>
{examples['test_comment'][i]}
</test_comment>"""
        
        # Assistant response is "Yes" or "No"
        assistant_msg = examples['violates_rule'][i]

        # Create messages list for chat template
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]


        formatted_text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )

        texts.append(formatted_text)

    return {"text": texts}

def build_dataset(tokenizer):
    """
    Build both train and test datasets using tokenizer chat template
    """
    df_train, df_test = load_data()
    
    train_dataset = Dataset.from_pandas(df_train)
    train_dataset = train_dataset.map(
        lambda examples: formatting_prompts_func(examples, tokenizer), 
        batched=True
    )
    
    test_dataset = Dataset.from_pandas(df_test)
    test_dataset = test_dataset.map(
        lambda examples: formatting_prompts_func(examples, tokenizer), 
        batched=True
    )
    
    return train_dataset, test_dataset

In [ ]:
%%writefile train.py

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, PeftModel
from transformers.utils import is_torch_bf16_gpu_available
from get_dataset import build_dataset
from config import LOCAL_MODEL_PATH, LORA_IN_PATH, LORA_OUT_PATH, RANK, MAX_SEQ_LENGTH, EPOCHS, TRAIN_DIR, MAX_ITER_STEPS, OUTPUT_PATH, RESUME_TRAINING, LORA_ALPHA
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# # ----------------------------
# # Load model & tokenizer
# # ----------------------------

model = AutoModelForCausalLM.from_pretrained(
    LOCAL_MODEL_PATH,
    #torch_dtype="auto",
    device_map="auto", 
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,      # Reduce CPU memory during loading
    trust_remote_code=True,
)

if RESUME_TRAINING: 
    model = PeftModel.from_pretrained(model, LORA_IN_PATH, is_trainable=True)


tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.gradient_checkpointing_enable()  # reduce memory usage    

# ----------------------------
# Build datasets
# ----------------------------
train_dataset, test_dataset = build_dataset(tokenizer)

# Add before training starts
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

# ----------------------------
# LoRA config
# ----------------------------
if RESUME_TRAINING:    
    lora_config = None       
else:    
    lora_config = LoraConfig(
        r=RANK,
        lora_alpha=LORA_ALPHA,
        lora_dropout=0.0,
        bias="none",
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj"
        ],
        task_type="CAUSAL_LM",
    )
    
# ----------------------------
# SFT config
# ----------------------------
sft_config = SFTConfig(
    output_dir=OUTPUT_PATH,
    num_train_epochs=EPOCHS,
    max_steps=MAX_ITER_STEPS,
    per_device_train_batch_size=4,       
    gradient_accumulation_steps=4,       
    max_length=min(MAX_SEQ_LENGTH, 4096),  

    optim="paged_adamw_8bit",
    learning_rate=1e-4,
    weight_decay=0.01,
    max_grad_norm=1.0,
        
    lr_scheduler_type="linear",
    warmup_ratio=0.05,
        
    bf16=is_torch_bf16_gpu_available(),
    fp16=not is_torch_bf16_gpu_available(),
    dataloader_pin_memory=True,
        
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},

    warmup_steps=5,
    logging_steps=10,
    eval_steps=3000,
    eval_strategy="epoch",
    save_strategy="no",
    save_total_limit=1,

    report_to="none",        
    packing=False,
    remove_unused_columns=False,
    dataset_text_field="text",

)

# ----------------------------
# Trainer
# ----------------------------
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=lora_config,
    args=sft_config,
)

trainer.train()
trainer.save_model(os.path.join(LORA_OUT_PATH, TRAIN_DIR))

torch.cuda.empty_cache()
print("success")

Overwriting train.py


In [20]:
%%writefile accelerate_config.yaml
# """
# ZeRO Stage 1: Optimizer State Partitioning (2-4 GPU)
# ZeRO Stage 2: + Gradient Partitioning (4-8 GPU)
# ZeRO Stage 3: + Parameter Partitioning (8+ GPU)
# """

compute_environment: LOCAL_MACHINE
debug: false
deepspeed_config:
  gradient_accumulation_steps: 4
  gradient_clipping: 1.0
  train_batch_size: 16
  train_micro_batch_size_per_gpu: 4
  
  zero_stage: 2
  offload_optimizer_device: none
  offload_param_device: none
  zero3_init_flag: false
  
  stage3_gather_16bit_weights_on_model_save: false
  stage3_max_live_parameters: 1e8
  stage3_max_reuse_distance: 1e8
  stage3_prefetch_bucket_size: 5e7
  stage3_param_persistence_threshold: 1e5
  
  zero_allow_untested_optimizer: true
  zero_force_ds_cpu_optimizer: false
  
  fp16:
    enabled: true
    loss_scale: 0
    initial_scale_power: 16
    loss_scale_window: 1000
    hysteresis: 2
    min_loss_scale: 1
  
distributed_type: DEEPSPEED
downcast_bf16: 'no'
dynamo_config:
  dynamo_backend: INDUCTOR
  dynamo_use_fullgraph: false
  dynamo_use_dynamic: false
enable_cpu_affinity: false
machine_rank: 0
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 1
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false


Overwriting accelerate_config.yaml


In [21]:
%%bash
mkdir -p ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [22]:
%%writefile upload.py
import os
import kagglehub
from config import LORA_OUT_PATH, TRAIN_DIR, MODEL_SLUG, VARIATION_SLUG
LOCAL_MODEL_DIR = os.path.join(LORA_OUT_PATH, TRAIN_DIR)

kagglehub.model_upload(
    handle=f"vinothkumarsekar89/{MODEL_SLUG}/transformers/{VARIATION_SLUG}",
    local_model_dir=LOCAL_MODEL_DIR,
    version_notes= TRAIN_DIR
)



Writing upload.py


In [23]:
%%writefile run.sh
./myvenv/bin/accelerate launch --config_file accelerate_config.yaml train.py
./myvenv/bin/python upload.py

Writing run.sh


In [1]:
%%bash
chmod +x run.sh
nohup bash run.sh > log.log 2>&1 &

In [4]:
%%bash
tail -f log.log